In [3]:
import pandas as pd
#Get raw data from all nba games from 1950 to 2017
nba_raw =  pd.read_csv('nba_games_all.csv',header='infer')

#Remove all data before the most recent team joined the league in 2002
recent_nba = nba_raw[nba_raw['season_year'] > 2002]

#Remove all rows that are not from the Regular Season
recent_nba = recent_nba[recent_nba['season_type'] == 'Regular Season']

#Remove all rows without win pct data
nba_full = recent_nba[pd.notnull(recent_nba['w_pct'])]

#Drop all columns that won't be needed since they are stats from that particular game
nba_full = nba_full.drop(['min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct',
                'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'season_year', 'season_type' ], axis=1)

#Sort the data by season and team_id
nba = nba_full.sort_values(by=['season', 'team_id'])

#Create second dataframe to store exact same data that we will merge to get comprehensive rows for each game
nba2 = nba.rename(columns={"team_id": "o_team_id", "is_home": "o_is_home", "wl": "o_wl", "w": "o_w", "l": "o_l", "w_pct": "o_w_pct",
                           "a_team_id": "o_a_team_id"})
nba2

,game_id,game_date,matchup,o_team_id,o_is_home,o_wl,o_w,o_l,o_w_pct,o_a_team_id,season
14289,20300128,2003-11-15,NJN @ ATL,1610612737,t,L,3.0,7.0,0.300,1610612751,2003-04
14290,20300102,2003-11-11,LAC vs. ATL,1610612737,f,L,2.0,6.0,0.250,1610612746,2003-04
14291,20300084,2003-11-08,POR vs. ATL,1610612737,f,L,1.0,5.0,0.167,1610612757,2003-04
14292,20300060,2003-11-05,GSW vs. ATL,1610612737,f,L,1.0,4.0,0.200,1610612744,2003-04
28936,20301176,2004-04-14,BOS vs. ATL,1610612737,f,L,28.0,54.0,0.341,1610612738,2003-04
...,...,...,...,...,...,...,...,...,...,...,...
106971,21700222,2017-11-17,CHI vs. CHA,1610612766,f,L,5.0,9.0,0.357,1610612741,2017-18
106972,21700126,2017-11-03,SAS vs. CHA,1610612766,f,L,5.0,4.0,0.556,1610612759,2017-18
106973,21700096,NaN,MEM vs. CHA,1610612766,f,W,4.0,3.0,0.571,1610612763,2017-18
106974,21700003,NaN,DET vs. CHA,1610612766,f,L,0.0,1.0,0.000,1610612765,2017-18


In [4]:
#Merge the 2 dataframes containing the same data to get all the data we need for each game in respective row
result = pd.merge(nba, nba2, how='inner', left_on=['game_id', 'a_team_id'], right_on=['game_id', 'o_team_id'],
         left_index=False, right_index=False, sort=False)
#Drop unneccessary columns
nba_c = result.drop(['game_date_x', 'game_date_y', 'a_team_id', 'matchup_y', 'season_y'], axis=1)

#Now we need to adjust all the w, l, w_pct values for each team in each game to be rolled back to
#before the stat was adjusted for the outcome of that particular game
nba_c

,game_id,matchup_x,team_id,is_home,wl,w,l,w_pct,season_x,o_team_id,o_is_home,o_wl,o_w,o_l,o_w_pct,o_a_team_id
0,20300128,NJN @ ATL,1610612737,t,L,3.0,7.0,0.300,2003-04,1610612751,f,W,5.0,5.0,0.500,1610612737
1,20300102,LAC vs. ATL,1610612737,f,L,2.0,6.0,0.250,2003-04,1610612746,t,W,2.0,2.0,0.500,1610612737
2,20300084,POR vs. ATL,1610612737,f,L,1.0,5.0,0.167,2003-04,1610612757,t,W,3.0,3.0,0.500,1610612737
3,20300060,GSW vs. ATL,1610612737,f,L,1.0,4.0,0.200,2003-04,1610612744,t,W,2.0,2.0,0.500,1610612737
4,20301176,BOS vs. ATL,1610612737,f,L,28.0,54.0,0.341,2003-04,1610612738,t,W,36.0,46.0,0.439,1610612737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35889,21700222,CHI vs. CHA,1610612766,f,L,5.0,9.0,0.357,2017-18,1610612741,t,W,3.0,10.0,0.231,1610612766
35890,21700126,SAS vs. CHA,1610612766,f,L,5.0,4.0,0.556,2017-18,1610612759,t,W,5.0,4.0,0.556,1610612766
35891,21700096,MEM vs. CHA,1610612766,f,W,4.0,3.0,0.571,2017-18,1610612763,t,L,5.0,2.0,0.714,1610612766
35892,21700003,DET vs. CHA,1610612766,f,L,0.0,1.0,0.000,2017-18,1610612765,t,W,1.0,0.0,1.000,1610612766


In [5]:
#Function for each row to adjust the w_pct for each team since the stat is currently for post-game
def adjust_pcts(row):
    new_row = row
    if row['wl'] == 'W':
        new_row['w'] = new_row['w']-1
        new_row['o_l'] = new_row['o_l']-1
        #w_pct first team
        if (new_row['w']+new_row['l']) == 0:
            new_row['w_pct'] = 0
        else:
            new_row['w_pct'] = new_row['w']/(new_row['w']+new_row['l'])
        #w_pct second team
        if (new_row['o_w']+new_row['o_l']) == 0:
            new_row['o_w_pct'] = 0
        else:
            new_row['o_w_pct'] = new_row['o_w']/(new_row['o_w']+new_row['o_l'])
    else:
        new_row['l'] = new_row['l']-1
        new_row['o_w'] = new_row['o_w']-1
        #w_pct first team
        if (new_row['w']+new_row['l']) == 0:
            new_row['w_pct'] = 0
        else:
            new_row['w_pct'] = new_row['w']/(new_row['w']+new_row['l'])
        #w_pct second team
        if (new_row['o_w']+new_row['o_l']) == 0:
            new_row['o_w_pct'] = 0
        else:
            new_row['o_w_pct'] = new_row['o_w']/(new_row['o_w']+new_row['o_l'])
    
    return new_row

#Apply the function
nba_u = nba_c.apply(adjust_pcts, axis=1)
nba_u

,game_id,matchup_x,team_id,is_home,wl,w,l,w_pct,season_x,o_team_id,o_is_home,o_wl,o_w,o_l,o_w_pct,o_a_team_id
0,20300128,NJN @ ATL,1610612737,t,L,3.0,6.0,0.333333,2003-04,1610612751,f,W,4.0,5.0,0.444444,1610612737
1,20300102,LAC vs. ATL,1610612737,f,L,2.0,5.0,0.285714,2003-04,1610612746,t,W,1.0,2.0,0.333333,1610612737
2,20300084,POR vs. ATL,1610612737,f,L,1.0,4.0,0.200000,2003-04,1610612757,t,W,2.0,3.0,0.400000,1610612737
3,20300060,GSW vs. ATL,1610612737,f,L,1.0,3.0,0.250000,2003-04,1610612744,t,W,1.0,2.0,0.333333,1610612737
4,20301176,BOS vs. ATL,1610612737,f,L,28.0,53.0,0.345679,2003-04,1610612738,t,W,35.0,46.0,0.432099,1610612737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35889,21700222,CHI vs. CHA,1610612766,f,L,5.0,8.0,0.384615,2017-18,1610612741,t,W,2.0,10.0,0.166667,1610612766
35890,21700126,SAS vs. CHA,1610612766,f,L,5.0,3.0,0.625000,2017-18,1610612759,t,W,4.0,4.0,0.500000,1610612766
35891,21700096,MEM vs. CHA,1610612766,f,W,3.0,3.0,0.500000,2017-18,1610612763,t,L,5.0,1.0,0.833333,1610612766
35892,21700003,DET vs. CHA,1610612766,f,L,0.0,0.0,0.000000,2017-18,1610612765,t,W,0.0,0.0,0.000000,1610612766


In [6]:
#Now we need to adjust the home/away stat to be a 1 or 0 and same for the W/L stat
def stats_to_numeric(row):
    new_row = row
    if row['is_home'] == 't':
        new_row['is_home'] = 1
    else:
        new_row['is_home'] = 0
    
    if row['wl'] == 'W':
        new_row['wl'] = 1
    else:
        new_row['wl'] = 0
    
    return new_row

#Apply the function
nba_u2 = nba_u.apply(stats_to_numeric, axis=1)
nba_u2

,game_id,matchup_x,team_id,is_home,wl,w,l,w_pct,season_x,o_team_id,o_is_home,o_wl,o_w,o_l,o_w_pct,o_a_team_id
0,20300128,NJN @ ATL,1610612737,1,0,3.0,6.0,0.333333,2003-04,1610612751,f,W,4.0,5.0,0.444444,1610612737
1,20300102,LAC vs. ATL,1610612737,0,0,2.0,5.0,0.285714,2003-04,1610612746,t,W,1.0,2.0,0.333333,1610612737
2,20300084,POR vs. ATL,1610612737,0,0,1.0,4.0,0.200000,2003-04,1610612757,t,W,2.0,3.0,0.400000,1610612737
3,20300060,GSW vs. ATL,1610612737,0,0,1.0,3.0,0.250000,2003-04,1610612744,t,W,1.0,2.0,0.333333,1610612737
4,20301176,BOS vs. ATL,1610612737,0,0,28.0,53.0,0.345679,2003-04,1610612738,t,W,35.0,46.0,0.432099,1610612737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35889,21700222,CHI vs. CHA,1610612766,0,0,5.0,8.0,0.384615,2017-18,1610612741,t,W,2.0,10.0,0.166667,1610612766
35890,21700126,SAS vs. CHA,1610612766,0,0,5.0,3.0,0.625000,2017-18,1610612759,t,W,4.0,4.0,0.500000,1610612766
35891,21700096,MEM vs. CHA,1610612766,0,1,3.0,3.0,0.500000,2017-18,1610612763,t,L,5.0,1.0,0.833333,1610612766
35892,21700003,DET vs. CHA,1610612766,0,0,0.0,0.0,0.000000,2017-18,1610612765,t,W,0.0,0.0,0.000000,1610612766


In [69]:
betting =  pd.read_csv('nba_betting_spread.csv',header='infer')
betting = betting[betting['book_name'] == '5Dimes']
betting = betting.drop(['book_id', 'price1', 'price2', 'book_name'], axis=1)
bet1 = betting.drop(['a_team_id', 'spread2'], axis=1)
bet2 = betting.drop(['team_id', 'spread1'], axis=1)
bet2 = bet2.rename(columns={"a_team_id": "team_id", "spread2": "spread1"})
bet_full = pd.concat([bet1,bet2])
result2 = pd.merge(nba_u2, bet_full, how='inner', left_on=['game_id', 'team_id'], right_on=['game_id', 'team_id'],
         left_index=False, right_index=False, sort=False)

spread_mean = result2['spread1'].mean()
spread_std = result2['spread1'].std(ddof=0)
result2['spread1'] = (result2['spread1'] - spread_mean)/spread_std
result2

,game_id,matchup_x,team_id,is_home,wl,w,l,w_pct,season_x,o_team_id,o_is_home,o_wl,o_w,o_l,o_w_pct,o_a_team_id,spread1
0,20600309,DEN @ ATL,1610612737,1,0,8.0,11.0,0.421053,2006-07,1610612743,f,W,11.0,7.0,0.611111,1610612737,1.189854
1,20600284,LAL vs. ATL,1610612737,0,0,8.0,9.0,0.470588,2006-07,1610612747,t,W,12.0,6.0,0.666667,1610612737,1.049857
2,20600269,DEN vs. ATL,1610612737,0,1,7.0,9.0,0.437500,2006-07,1610612743,t,L,10.0,5.0,0.666667,1610612737,1.679841
3,20600186,ORL vs. ATL,1610612737,0,0,5.0,5.0,0.500000,2006-07,1610612753,t,W,9.0,4.0,0.692308,1610612737,1.049857
4,20600161,DET vs. ATL,1610612737,0,0,4.0,4.0,0.500000,2006-07,1610612765,t,W,6.0,5.0,0.545455,1610612737,0.839863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27727,21700687,SAC @ CHA,1610612766,1,1,18.0,26.0,0.409091,2017-18,1610612758,f,L,13.0,32.0,0.288889,1610612766,-1.400080
27728,21700277,SAS @ CHA,1610612766,1,0,8.0,10.0,0.444444,2017-18,1610612759,f,W,11.0,7.0,0.611111,1610612766,-0.210111
27729,21700222,CHI vs. CHA,1610612766,0,0,5.0,8.0,0.384615,2017-18,1610612741,t,W,2.0,10.0,0.166667,1610612766,-0.910093
27730,21700126,SAS vs. CHA,1610612766,0,0,5.0,3.0,0.625000,2017-18,1610612759,t,W,4.0,4.0,0.500000,1610612766,0.629868


In [70]:
#Trim for just the numerical values now
nba_f = result2.drop(['game_id', 'matchup_x', 'team_id', 'w', 'l', 'season_x', 'o_team_id',
                     'o_is_home', 'o_wl', 'o_w', 'o_l', 'o_a_team_id'], axis=1)
y = nba_f['wl'].values
nba_f = nba_f.drop(['wl'], axis=1)
X = nba_f.values
X

array([[ 1.        ,  0.42105263,  0.61111111,  1.18985361],
       [ 0.        ,  0.47058824,  0.66666667,  1.04985717],
       [ 0.        ,  0.4375    ,  0.66666667,  1.67984114],
       ...,
       [ 0.        ,  0.38461538,  0.16666667, -0.91009295],
       [ 0.        ,  0.625     ,  0.5       ,  0.62986786],
       [ 1.        ,  0.4375    ,  0.58823529, -0.42010542]])

In [82]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Split the data to testing and training sets
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

#Train the model
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

#Predict using the test data
y_pred = clf.predict(X_test)
#Get accuracy score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.6938374125874126


In [77]:
#Example prediction
adj_spread = (1.5 - spread_mean)/spread_std #Replace 1.5 with point spread from betting company
y_pred = clf.predict([[0, 0.7727, 0.624, adj_spread]]) #1/0 for home/away, team winning pct, opposing winning pct
y_pred

array([0])

In [78]:
#Model coefficient values
clf.coef_

array([[-0.1506417 , -0.30819768,  0.39375691, -1.14193872]])